# ПРИМЕР ВОПРОСНО-ОТВЕТНОЙ СИСТЕМЫ НА БАЗЕ SEQUENCE TO SEQUENCE АРХИТЕКТУРЫ

Данильченко Вадим

-------------------------------------------

In [1]:
# загрузим библиотеки
from __future__ import print_function

import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import joblib
from tqdm import trange

import warnings
warnings.filterwarnings('ignore')

подготовим данные для обучения

In [2]:
# зададим параметры модели
batch_size = 64  # размер батча
epochs = 50  # количество эпох обучения
latent_dim = 512  # укажем количество нейронов для енкодера
num_samples = 30000  # количество примеров для обучения

In [3]:
# загрузим данные
data = joblib.load('./dialogues_prepared.pkl')
data.shape
questions = data.participant_1.tolist()
answers = data.participant_2.tolist()

if len(questions)>num_samples:
    questions = questions[:num_samples]
    answers = answers[:num_samples]

print(questions[:2])
print(answers[:2])

['Привет) расскажи о себе', 'Что читаешь? Мне нравится классика . Я тоже люблю пообщаться']
['Привет) под вкусный кофеек настроение поболтать появилось )', 'Люблю животных, просто обожаю, как и свою работу) . Я фантастику люблю']


In [4]:
# подготовим последовательность токенов для каждого примера
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

max_encoder_seq_length = 100
max_decoder_seq_length = 100

# '\t' - начало последовательности 
# '\n' - окончание
for line in trange(len(questions)):
    input_text, target_text = questions[line][:max_encoder_seq_length-2], answers[line][:max_decoder_seq_length-2]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


100%|█████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 75382.80it/s]


Number of samples: 30000
Number of unique input tokens: 293
Number of unique output tokens: 308
Max sequence length for inputs: 100
Max sequence length for outputs: 100


In [5]:
# векторизуем данные
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data - целевая последовательность, содержит токены старта и конца последовательности
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data - сдвиг на один токен от decoder_input_data
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [14]:
{k:v for i,(k,v) in enumerate(input_token_index.items()) if i<20}

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19}

In [15]:
{k:v for i,(k,v) in enumerate(target_token_index.items()) if i<20}

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 '+': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '/': 15,
 '0': 16,
 '1': 17,
 '2': 18,
 '3': 19}

зададим encoder-decoder архитектуру нейросети

In [6]:
# опишем энкодер
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# на выходе энкодера оставляем только состояния
encoder_states = [state_h, state_c]

# опишем декодер, на вход которого будут приходить целевая последовательность id токенов и состояния энкодера
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
# компилируем модель
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 293)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 308)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 512), (None, 1650688     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 512),  1681408     input_2[0][0]                    
                                                                 lstm[0][1]                   

In [35]:
# запускаем обучение
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2, 
          verbose=2)

Epoch 1/50
375/375 - 79s - loss: 0.3060 - accuracy: 0.9072 - val_loss: 0.8329 - val_accuracy: 0.8134
Epoch 2/50
375/375 - 78s - loss: 0.3041 - accuracy: 0.9077 - val_loss: 0.8373 - val_accuracy: 0.8132
Epoch 3/50
375/375 - 77s - loss: 0.3026 - accuracy: 0.9081 - val_loss: 0.8399 - val_accuracy: 0.8136
Epoch 4/50
375/375 - 78s - loss: 0.3007 - accuracy: 0.9085 - val_loss: 0.8449 - val_accuracy: 0.8132
Epoch 5/50
375/375 - 78s - loss: 0.2991 - accuracy: 0.9088 - val_loss: 0.8474 - val_accuracy: 0.8142
Epoch 6/50
375/375 - 81s - loss: 0.2979 - accuracy: 0.9091 - val_loss: 0.8561 - val_accuracy: 0.8126
Epoch 7/50
375/375 - 82s - loss: 0.2961 - accuracy: 0.9098 - val_loss: 0.8585 - val_accuracy: 0.8125
Epoch 8/50
375/375 - 80s - loss: 0.2945 - accuracy: 0.9101 - val_loss: 0.8585 - val_accuracy: 0.8130
Epoch 9/50
375/375 - 83s - loss: 0.2934 - accuracy: 0.9102 - val_loss: 0.8574 - val_accuracy: 0.8136
Epoch 10/50
375/375 - 86s - loss: 0.2915 - accuracy: 0.9110 - val_loss: 0.8666 - val_accura

In [25]:
# сохраним модель
model.save('seq2seq_talks.h5')

In [26]:
# шаги инференса модели:
# 1. пропустим через энкодер и получим начальное состояние декодера
# 2. запустим декодер с полученным начальным состоянием и целевым токеном 
# старта последовательности '\t', выходом будет следующий целевой токен
# 3. получим состояния и используем их вместе с полученным токеном для следующей итерации
# 4. прекратим при предсказании токена конца последовательности '\n' 

# задаем модели
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [27]:
# создаем словарь индекс токена в токен
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [28]:
def decode_sequence(input_seq):
    # 1. выход энкодера будет начальным состоянием декодера
    states_value = encoder_model.predict(input_seq)

    # генерируем целевой начальный таргет для декодера '\t'
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # пункты 2-4
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 2. декодер с начальным состоянием и целевым таргетом 
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 3. полученный токен
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 4. условия выхода из цикла
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # обновим целевой токен
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # обновим состояния
        states_value = [h, c]

    return decoded_sentence

In [70]:
# получим несколько последовательностей на основе датасета
for seq_index in range(52, 57):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Конечно)
Decoded sentence: 

-
Input sentence: Привет
Decoded sentence: Привет

-
Input sentence: Ой, слушай, довольно таки неплохо. Гораздо лучше, чем днём . А твои как?
Decoded sentence: А я люблю путешествовать . А ты?

-
Input sentence: В свободное от работы время я люблю готовиться всякие вкусности. В детстве мама меня учила готовит
Decoded sentence: Я тоже люблю покушать, как говорят домосей )) расскажи про своим любимом места помоглешь и умею лю 

-
Input sentence: Ох, соболезную ..
Decoded sentence: Нет, я один живу . Можешь заняться в театр . Ау



In [46]:
# функция для использования в модели произвольного текста
def make_encoder_input(text):
    encoder_input_text = np.zeros([1, max_encoder_seq_length, num_encoder_tokens])
    phrase = text
    for i in trange(len(phrase)):
        if i>max_encoder_seq_length:
            break
        print(phrase[i])
        if phrase[i] not in input_characters:
            continue
        encoder_input_text[0, i, input_characters.index(phrase[i])] = 1
    return encoder_input_text

In [58]:
#####################################################################
input_sentence = 'умеешь говорить?'
input_seq = make_encoder_input(input_sentence)
decoded_sentence = decode_sequence(input_seq)
print('-')
print('Input sentence:', input_sentence)
print('Decoded sentence:', decoded_sentence)

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

у
м
е
е
ш
ь
 
г
о
в
о
р
и
т
ь
?


100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 573.19it/s]


-
Input sentence: умеешь говорить?
Decoded sentence: Нет, я люблю путешествовать . А ты?



-------------------------------
Выводы: результаты довольно интересные, но еще есть что поделать для улучшения:
1. увеличить датасет
2. посмотреть на диалоги, видно много разных смыслов в одном предложении, по-хорошему бы их разделить
3. поиграть с архитектурой, убрать дэльту между точностью на обучении и тесте
4. добавить эпох обучения